<a href="https://colab.research.google.com/github/HWP-Wilson/Classificador_de_Tweet-s_com_TFHub/blob/main/Classificador__de_tweets_com_TFHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificador de Tweet  com modelo  TF Hub
@helanowilson@ufc.br




In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import json
import pickle
import urllib

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import wordninja
import textblob
import re

import warnings
warnings.filterwarnings("ignore")



from nltk.tokenize import TweetTokenizer

print(tf.__version__)

2.4.1


## Dataset

Temos aproxidamente 25 mil tweets classificados em Discurso de ódio, linguagem ofensiva ou Neutro. 

In [ ]:
##Dataset
data = pd.read_csv('Dados/labeled_data.csv')
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


## Ajustes nos dados para implementação do modelo

In [ ]:
##Criação de dicionário para classificação
Classe = {0: ['Discurso de ódio'], 1: ['Linguagem ofensiva'], 2: ['Neutro']}
data['classe']= data['class'].map(Classe)

In [ ]:
# Excluindo da descrição os números, informações julgadas irrelevantes para a classificação.
data['tweet_novo'] = data['tweet'].str.replace('[0-9]+', '', regex=True).copy()
# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
data['tweet_novo'] = data['tweet_novo'].str.replace('[,.:;!?]+', ' ', regex=True).copy()
# Excluindo da descrição caracteres especiais, informações julgadas irrelevantes para a classificação.
data['tweet_novo'] = data['tweet_novo'].str.replace("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", regex=True).copy()

# Colocando todos os caracteres em caixa baixa.
data['tweet_novo'] = data['tweet_novo'].str.lower().copy()


In [ ]:
# Função para retirar stop words
punct = list(string.punctuation)
stop_words = stopwords.words('english')
additional_stop_words = ['RT', 'rt', 'via', '...', 'http', 'twitpic',
'tinyurl' ,'www']
stopword_list = punct + stop_words + additional_stop_words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stopword_list]
  text = TreebankWordDetokenizer().detokenize(stop)
  return text
# Eliminando as stop words
data['tweet_organizado'] = data['tweet_novo'].apply(tokenize_df).copy()
data.head(2)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,classe,tweet_novo,tweet_organizado
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,[Neutro],rt as a woman you shouldn't complain abou...,womann't complain cleaning house amp man alway...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,[Linguagem ofensiva],rt boy dats cold tyga dwn bad for cuffin ...,boy dats cold tyga dwn bad cuffin dat hoe st p...


In [ ]:
# Separando palavras juntas
data['text_split'] = data['tweet_organizado'].apply(wordninja.split)
data['text_novo'] = data['text_split'].apply(TreebankWordDetokenizer().detokenize)
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,classe,tweet_novo,tweet_organizado,text_split,text_novo
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,[Neutro],rt as a woman you shouldn't complain abou...,womann't complain cleaning house amp man alway...,"[woman, n, ', t, complain, cleaning, house, am...",woman n' t complain cleaning house amp man alw...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,[Linguagem ofensiva],rt boy dats cold tyga dwn bad for cuffin ...,boy dats cold tyga dwn bad cuffin dat hoe st p...,"[boy, dat, s, cold, tyga, dw, n, bad, cuff, in...",boy dat s cold tyga dw n bad cuff in dat hoe s...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,[Linguagem ofensiva],rt dawg rt you ever fuck a bitch and s...,dawg ever fuck bitch start cry confused shit,"[daw, g, ever, fuck, bitch, start, cry, confus...",daw g ever fuck bitch start cry confused shit
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,[Linguagem ofensiva],rt _g_anderson _based she look like a tranny,_g_anderson _based look like tranny,"[g, anderson, based, look, like, tr, an, ny]",g anderson based look like tr an ny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,[Linguagem ofensiva],rt the shit you hear about me might be tr...,shit hear might true might faker bitch told ya,"[shit, hear, might, true, might, faker, bitch,...",shit hear might true might faker bitch told ya


In [ ]:
#corrigindo palavras incorretas
data['tweet_definitivo'] = data['text_novo'].apply(textblob.TextBlob).apply(textblob.TextBlob.correct).apply(str)
data.head(2)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,classe,tweet_novo,tweet_organizado,text_split,text_novo,tweet_definitivo
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,[Neutro],rt as a woman you shouldn't complain abou...,womann't complain cleaning house amp man alway...,"[woman, n, ', t, complain, cleaning, house, am...",woman n' t complain cleaning house amp man alw...,woman n' t complain cleaning house amp man alw...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,[Linguagem ofensiva],rt boy dats cold tyga dwn bad for cuffin ...,boy dats cold tyga dwn bad cuffin dat hoe st p...,"[boy, dat, s, cold, tyga, dw, n, bad, cuff, in...",boy dat s cold tyga dw n bad cuff in dat hoe s...,boy dat s cold tea do n bad cuff in dat he st ...


In [ ]:
descriptions = data['tweet_definitivo']
genres =data['classe']

In [ ]:
classification=genres.to_numpy()
classification

array([list(['Neutro']), list(['Linguagem ofensiva']),
       list(['Linguagem ofensiva']), ..., list(['Linguagem ofensiva']),
       list(['Linguagem ofensiva']), list(['Neutro'])], dtype=object)

In [ ]:
train_size = int(len(descriptions) * .8)

train_descriptions = descriptions[:train_size].astype('str')
train_genres = genres[:train_size]

test_descriptions = descriptions[train_size:].astype('str')
test_genres = genres[train_size:]

In [ ]:
test_genres.head()

19826                [Neutro]
19827    [Linguagem ofensiva]
19828    [Linguagem ofensiva]
19829    [Linguagem ofensiva]
19830                [Neutro]
Name: classe, dtype: object

### Formatação das Labels, criando vetores. 

In [ ]:
encoder = MultiLabelBinarizer()
encoder.fit_transform(train_genres)
train_encoded = encoder.transform(train_genres)
test_encoded = encoder.transform(test_genres)
num_classes = len(encoder.classes_)

# Printando as possibilidades de classifacação para cada tweet e para o primeiro tweet.
print(encoder.classes_)
print(train_encoded[0])

['Discurso de ódio' 'Linguagem ofensiva' 'Neutro']
[0 0 1]


### Criando TF Hub embedding layer
Vetorização de todo o texto na coluna descrição. 


In [ ]:
description_embeddings = hub.text_embedding_column("descriptions", module_spec="https://tfhub.dev/google/universal-sentence-encoder/2", trainable=False)


## Instanciando o modelo com DNNEstimator 
O primeiro parâmetro que passamos para nosso DNNEstimator é chamado de cabeça e define o tipo de rótulos que nosso modelo deve esperar. Como queremos que nosso modelo produza vários rótulos, usaremos multi_label_head aqui. Em seguida, converteremos nossos recursos e rótulos em matrizes numpy e instanciaremos nosso Estimator. `batch_size` e` num_epochs` são hiperparâmetros  

In [ ]:
multi_label_head = tf.estimator.MultiLabelHead(
    num_classes, weight_column=None, thresholds=None, label_vocabulary=None,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

In [ ]:
features = {
  "descriptions": np.array(train_descriptions).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=1024, num_epochs=25)
estimator = tf.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings])


INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Jamili\\AppData\\Local\\Temp\\tmpnxd7fuzz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Jamili\\AppData\\Local\\Temp\\tmpnxd7fuzz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Treinando e executando o modelo


In [ ]:
estimator.train(input_fn=train_input_fn)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


...

INFO:tensorflow:Saving checkpoints for 485 into C:\Users\Jamili\AppData\Local\Temp\tmpnxd7fuzz\model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 485...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 485...


INFO:tensorflow:Loss for final step: 0.604857.


INFO:tensorflow:Loss for final step: 0.604857.


In [ ]:
# Definindo as entradas e avaliador
eval_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn({"descriptions": np.array(test_descriptions).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 485: auc = 0.8805411, auc_precision_recall = 0.7594211, average_loss = 0.60455483, global_step = 485, loss = 0.60451764


INFO:tensorflow:Saving dict for global step 485: auc = 0.8805411, auc_precision_recall = 0.7594211, average_loss = 0.60455483, global_step = 485, loss = 0.60451764


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 485: C:\Users\Jamili\AppData\Local\Temp\tmpnxd7fuzz\model.ckpt-485


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 485: C:\Users\Jamili\AppData\Local\Temp\tmpnxd7fuzz\model.ckpt-485


{'auc': 0.8805411,
 'auc_precision_recall': 0.7594211,
 'average_loss': 0.60455483,
 'loss': 0.60451764,
 'global_step': 485}

## Predições com novos dados


In [ ]:
# Testando o modelo com dados já existentes
raw_test = [
    "As a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out.", # Neutro
    "bitch nigga miss me with it.", # Linguagem ofensiva
    "playing football with friends.", # Neutro
]



In [ ]:
# Gerando predições
predict_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn({"descriptions": np.array(raw_test).astype(np.str)}, shuffle=False)
results = estimator.predict(predict_input_fn)

In [ ]:
# Mostrando as predições
for tweet_genres in results:
  top_2 = tweet_genres['probabilities'].argsort()[-2:][::-1]
  for genre in top_2:
    text_genre = encoder.classes_[genre]
    print(text_genre + ': ' + str(round(tweet_genres['probabilities'][genre] * 100, 2)) + '%')
  print('')

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Linguagem ofensiva: 54.28%
Neutro: 46.6%

Linguagem ofensiva: 55.29%
Neutro: 46.72%

Linguagem ofensiva: 54.48%
Neutro: 47.66%

